# Tritement des données

Importation des modules

In [185]:
import pandas as pd  
import numpy as np, pandas as pd, matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import mannwhitneyu, norm
import chardet
import unicodedata


## Importation des bases de données

In [186]:

# USAGERS impliqués dans l'accident
base_us = pd.read_csv("usagers-2021.csv", delimiter=",", low_memory=False)

In [187]:
base_veh = pd.read_csv("vehicules-2021.csv", delimiter=";", encoding="latin1", low_memory=False)

In [188]:
# Charger le fichier qui décrit les circonstances générales de l’accident
base_car = pd.read_excel("caracteristiques.xlsx")    

### Analayse de la base des caracteristiques

In [189]:
base_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56530 entries, 0 to 56529
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num_Acc  56530 non-null  int64 
 1   jour     56530 non-null  int64 
 2   mois     56530 non-null  int64 
 3   an       56530 non-null  int64 
 4   hrmn     56530 non-null  object
 5   lum      56530 non-null  int64 
 6   com      56530 non-null  object
 7   agg      56530 non-null  int64 
 8   int      56530 non-null  int64 
 9   atm      56530 non-null  int64 
 10  col      56530 non-null  int64 
 11  adr      55957 non-null  object
 12  lat      56530 non-null  object
 13  long     56530 non-null  object
dtypes: int64(9), object(5)
memory usage: 6.0+ MB


In [190]:
print(" la base des caracteristiques contient", sum(base_car.duplicated('Num_Acc')), "doublons")

 la base des caracteristiques contient 11 doublons


In [191]:
#pour compter les doublon pur/parfaits
print("la base des caracteristiques contient", sum(base_car.duplicated()),'doublon purs')

la base des caracteristiques contient 11 doublon purs


In [192]:
# Identifier les lignes dupliquées
duplicated = base_car[base_car.duplicated( keep=False)]
# Trier les lignes dupliquées par ordre croissant de 'Num_Acc'
duplicated = duplicated.sort_values(by='Num_Acc')
# Afficher les lignes dupliquées triées
duplicated 


,Num_Acc,jour,mois,an,hrmn,lum,com,agg,int,atm,col,adr,lat,long
33687,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
6317,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
25206,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
19025,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
32639,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
420,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
45121,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
50662,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
2994,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"
55470,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"


On constate qu'en suivant Num_Acc on a 11 doublons identiques pour chacunes de nos variables que l'on peut supprimer.
Nous allons supprimer la dernière occurrence car elle ne fait aucune différence avec la preemière pour chacunes de nos observations

In [193]:
base_car = base_car.drop_duplicates(subset='Num_Acc', keep='first')
base_car

,Num_Acc,jour,mois,an,hrmn,lum,com,agg,int,atm,col,adr,lat,long
0,202100037733,28,5,2021,14:15,1,CC_97407,2,2,1,3,JEAN MOULIN (RUE) - RIV. DES GALETS,"-20,95888","55,31743"
1,202100049382,2,3,2021,10:59,1,CC_63113,2,1,1,2,SAINT ROBERT (RUE),"45,79137","3,1162"
2,202100038891,20,5,2021,16:50,1,CC_88160,2,2,1,1,POISSOMPRE (FG DE),"48,18261","6,47218"
3,202100028616,14,7,2021,19:45,1,CC_80021,2,1,1,3,CAGNARD (RUE),"49,90355","2,2872"
4,202100033509,17,6,2021,23:40,3,CC_13214,2,1,1,1,A7-Sens Lyon vers Marseille,"43,34031","5,37689"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56525,202100026753,28,7,2021,17:10,1,CC_38474,2,2,1,3,Rue du Vinay,"45,204957","5,667561"
56526,202100019814,10,9,2021,06:15,3,CC_77251,1,1,1,7,RN 104,"48,631","2,533"
56527,202100016048,29,9,2021,08:15,1,CC_78524,1,1,1,3,A13,"48,841059","2,106997"
56528,202100039830,15,5,2021,06:45,2,CC_13212,2,2,1,3,PIERRE CHEVALIER (avenue),"43,2953","5,4254"


### Analyse de la base usagers

In [194]:
base_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129189 entries, 0 to 129188
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      129189 non-null  int64  
 1   id_vehicule  129189 non-null  object 
 2   num_veh      129189 non-null  object 
 3   place        129189 non-null  int64  
 4   catu         129189 non-null  int64  
 5   grav         129189 non-null  int64  
 6   sexe         129189 non-null  int64  
 7   an_nais      126122 non-null  float64
 8   trajet       129189 non-null  int64  
 9   secu1        129189 non-null  int64  
 10  secu2        129189 non-null  int64  
 11  secu3        129189 non-null  int64  
 12  locp         129189 non-null  int64  
 13  actp         129189 non-null  object 
 14  etatp        129189 non-null  int64  
dtypes: float64(1), int64(11), object(3)
memory usage: 14.8+ MB


In [195]:
print("la base des usagers contient", sum(base_us.duplicated('Num_Acc')), "doublons")

la base des usagers contient 72668 doublons


In [196]:
#pour compter les doublon pur/parfaits
print("la base des usagers contient", sum(base_us.duplicated()),'doublons purs')

la base des usagers contient 33 doublons purs


In [197]:
base_us = base_us.drop_duplicates( keep='first')
base_us

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202100021165,163Â 830,A01,9,2,4,1,1948.0,0,1,-1,-1,0,0,-1
1,202100020281,165Â 383,A01,1,1,1,2,1965.0,1,1,8,-1,-1,-1,-1
2,202100047336,117Â 153,A01,1,1,4,2,1972.0,0,2,0,-1,-1,-1,-1
3,202100008918,185Â 779,B01,1,1,4,1,1985.0,9,8,0,-1,-1,-1,-1
4,202100009175,185Â 315,A01,1,1,3,1,1960.0,5,1,-1,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129184,202100018230,169Â 018,D01,1,1,1,2,1982.0,5,1,-1,-1,0,0,-1
129185,202100050268,111Â 866,B01,2,2,4,2,1942.0,1,1,-1,-1,0,0,-1
129186,202100056519,163Â 830,A01,9,2,4,1,1948.0,0,1,-1,-1,0,0,-1
129187,202100056520,165Â 383,A01,1,1,1,2,1965.0,1,1,8,-1,-1,-1,-1


In [198]:
#pour compter les doublon pur/parfaits
print("la base des usagers contient", sum(base_us.duplicated()),'doublon purs')

la base des usagers contient 0 doublon purs


In [199]:
base_us.sort_values(by="Num_Acc", ascending=True).head(10)

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
24992,202100000001,201Â 765,A01,1,1,1,1,1978.0,1,1,-1,-1,0,0,-1
32123,202100000001,201Â 764,B01,1,1,3,1,2000.0,1,0,9,-1,0,0,-1
45309,202100000002,201Â 762,A01,1,1,4,1,1983.0,0,1,-1,-1,0,0,-1
14610,202100000002,201Â 763,B01,1,1,3,1,1993.0,0,1,-1,-1,0,0,-1
88804,202100000003,201Â 761,A01,10,3,3,2,1959.0,4,0,-1,-1,3,3,1
23858,202100000003,201Â 761,A01,1,1,1,1,1995.0,1,1,0,-1,0,0,-1
113992,202100000004,201Â 759,D01,1,1,2,2,2014.0,5,0,-1,-1,-1,-1,-1
104179,202100000004,201Â 758,A01,1,1,1,1,2000.0,-1,-1,0,-1,-1,-1,-1
53845,202100000005,201Â 755,Z01,1,1,-1,-1,NaN,-1,8,-1,-1,-1,-1,-1
89513,202100000005,201Â 754,A01,1,1,4,2,1997.0,1,1,-1,-1,-1,-1,-1


cela est possible car plusieurs usagers peuvent être associés au même accident : Num_Acc

### Analyse au niveau de la base du Véhicule

In [200]:
base_veh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97365 entries, 0 to 97364
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ï»¿id_vehicule  97365 non-null  object 
 1   num_veh         97365 non-null  object 
 2   senc            97365 non-null  int64  
 3   catv            97365 non-null  int64  
 4   obs             97365 non-null  int64  
 5   obsm            97365 non-null  int64  
 6   choc            97365 non-null  int64  
 7   manv            97365 non-null  int64  
 8   motor           97365 non-null  int64  
 9   occutc          744 non-null    float64
dtypes: float64(1), int64(7), object(2)
memory usage: 7.4+ MB


In [201]:
# Supprimer les caractères spéciaux des noms de colonnes
base_veh.columns = base_veh.columns.str.encode('ascii', 'ignore').str.decode('ascii')


In [202]:
print(base_veh["occutc"].describe())  # Statistiques descriptives
print("Nombre de valeurs manquantes :", base_veh["occutc"].isna().sum())  # Compte les NaN
missing_percentage = base_veh["occutc"].isna().mean() * 100
print(f"Pourcentage de valeurs manquantes : {missing_percentage:.2f}%")


count    744.000000
mean       1.653226
std        1.951130
min        0.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       28.000000
Name: occutc, dtype: float64
Nombre de valeurs manquantes : 96621
Pourcentage de valeurs manquantes : 99.24%


##### Fusionner avec la base des usagers avec la base des caractéristique

In [203]:
print(base_us[['id_vehicule', 'num_veh']].drop_duplicates().head())
print(base_veh[['id_vehicule', 'num_veh']].drop_duplicates().head())


  id_vehicule num_veh
0    163Â 830     A01
1    165Â 383     A01
2    117Â 153     A01
3    185Â 779     B01
4    185Â 315     A01
  id_vehicule num_veh
0    129åÊ983     A01
1    192åÊ850     B01
2    152åÊ776     A01
3    193åÊ398     A01
4    178åÊ605     A01


La clé de fusion de base_veh et base_us en termes de "id_vehicule" affiche de valeur différente avec des caractères anormals. Nous alons corriger ces caractères

In [204]:
def clean_string(s):
    if isinstance(s, str):
        s = unicodedata.normalize('NFKD', s)  # décompose les caractères accentués
        s = s.encode('ascii', 'ignore').decode('ascii')  # supprime tout ce qui n'est pas ASCII
        return s.strip()
    return s

# Appliquer sur les colonnes des deux bases
for df in [base_us, base_veh]:
    df['id_vehicule'] = df['id_vehicule'].astype(str).apply(clean_string)
    df['num_veh'] = df['num_veh'].astype(str).apply(clean_string)

base_veh["id_vehicule"] = base_veh["id_vehicule"].str.replace("aE", "A ")


In [205]:
# Fusion base_us et base_veh
us_veh = base_us.merge(base_veh, on=['id_vehicule', 'num_veh'], how='left')

# Fusion avec base_car
base_final = us_veh.merge(base_car, on='Num_Acc', how='left')

In [206]:
# Compter le nombre d’usagers par accident 
victimes_par_accident = base_us.groupby("Num_Acc").size().reset_index(name="nb_usagers")
victimes_par_accident

,Num_Acc,nb_usagers
0,202100000001,2
1,202100000002,2
2,202100000003,2
3,202100000004,2
4,202100000005,2
...,...,...
56516,202100056517,1
56517,202100056518,2
56518,202100056519,1
56519,202100056520,1


In [207]:
base_final= base_final.merge(victimes_par_accident, on="Num_Acc", how="left")

In [208]:
base_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129156 entries, 0 to 129155
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      129156 non-null  int64  
 1   id_vehicule  129156 non-null  object 
 2   num_veh      129156 non-null  object 
 3   place        129156 non-null  int64  
 4   catu         129156 non-null  int64  
 5   grav         129156 non-null  int64  
 6   sexe         129156 non-null  int64  
 7   an_nais      126089 non-null  float64
 8   trajet       129156 non-null  int64  
 9   secu1        129156 non-null  int64  
 10  secu2        129156 non-null  int64  
 11  secu3        129156 non-null  int64  
 12  locp         129156 non-null  int64  
 13  actp         129156 non-null  object 
 14  etatp        129156 non-null  int64  
 15  senc         129156 non-null  int64  
 16  catv         129156 non-null  int64  
 17  obs          129156 non-null  int64  
 18  obsm         129156 non-

#### Vérifions les variables incohérentes

In [209]:
base_final["sexe"].value_counts()

sexe
 1    86192
 2    39892
-1     3062
 3       10
Name: count, dtype: int64

In [210]:
# Filtrer la base pour ne garder que les valeurs valides (1 = Homme, 2 = Femme)
base_final = base_final[base_final["sexe"].isin([1, 2])]


Pour la suite nous supposons que une valeur NaN peut être considéré comme une valeur inconnue et être remplacée par "0" : indeterminé

Le traitement sera appliqué à : trajet,secu1, secu2, secu3, locp, actp, etatp, senc, catv, obs, obsm , choc

In [211]:
# Calculer le mode de chaque colonne
modes = base_final.mode().iloc[0]  # Prend la première valeur en cas de multimodalité

# Appliquer la transformation colonne par colonne
for col in base_final.columns:
    base_final[col] = base_final[col].replace(-1, modes[col]).replace(0, "Aucun")


In [212]:
base_final["choc"].value_counts()

choc
1        45624
3        18773
2        16490
4        12798
8         8204
7         7260
Aucun     7073
6         4426
5         3430
9         2006
Name: count, dtype: int64

In [213]:
base_final['hrmn'] = base_final['hrmn'].fillna("0000")  # Remplace les valeurs manquantes


traitement de la variable commune

In [214]:
# Supprimer "CC_" au début de chaque valeur
base_final['com'] = base_final['com'].str.replace('CC_', '', regex=True)

# Calcul du mode de la variable 'com'
mode_com = base_final['com'].mode()[0] 
# Remplacement des valeurs manquantes
base_final['com'].fillna(mode_com, inplace=True)

# Vérification du résultat
print(f"Valeur utilisée pour le remplissage : {mode_com}")
print(base_final['com'].isna().sum()) 

Valeur utilisée pour le remplissage : 75116
0


C:\Users\SCD UM\AppData\Local\Temp\ipykernel_132\3923965944.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_final['com'].fillna(mode_com, inplace=True)


générons la variable département

In [215]:
# Extraction du département à partir du code commune
base_final['dep'] = base_final['com'].astype(str).str[:2]


In [216]:
# Analyse de l’âge
base_final["age"] = 2021 - base_final["an_nais"]
base_final = base_final[(base_final["age"] <= 100)]

# Catégorie du véhicule (catv)
base_final['catv'] = base_final['catv'].replace({1: 'Bicyclette',2: 'Cyclomoteur <50cc',3: 'Voiturette',7: 'VL seul',
    10: 'VU ≤ 3.5T',13: 'PL 3.5-7.5T',14: 'PL > 7.5T',15: 'PL + remorque',16: 'Tracteur routier seul',
    17: 'Tracteur + semi-remorque',20: 'Engin spécial',21: 'Tracteur agricole',30: 'Scooter <50cc',31: 'Moto ≤125cc',
    33: 'Moto >125cc',37: 'Autobus',38: 'Autocar',40: 'Tramway',80: 'VAE',99: 'Autre'
})

In [217]:
# Reclassification complète et cohérente de catv
base_final['catv'] = base_final['catv'].replace({
    0:'Aucun',
    # Deux-roues non motorisés
    1: 'Deux-roues non motorisés',80: 'Deux-roues non motorisés',# Bicyclette et  VAE
    
    # Deux-roues motorisés (Scooter <50cc, Cyclomoteur <50cc,Moto ≤125cc,, Moto >125cc, Trois-roues motorisé,Quad,Trottinette, gyropode, etc.)
    2: 'Deux-roues motorisés',30: 'Deux-roues motorisés', 31: 'Deux-roues motorisés',33: 'Deux-roues motorisés',
    35: 'Deux-roues motorisés',36: 'Deux-roues motorisés', 60: 'Deux-roues motorisés',
    
    # Véhicules légers (VP) : Voiturette,  VL seul, Camping-car
    3: 'Véhicules légers', 7: 'Véhicules légers',34: 'Véhicules légers',
    
    # Véhicules utilitaires légers / pro
    10: 'Véhicules légers pro',# VU ≤ 3.5T
    
    # Poids lourds :PL 3.5-7.5T, PL > 7.5T, PL + remorque, Tracteur routier seul, Tracteur + semi-remorque
    13: 'Poids lourds',14: 'Poids lourds',15: 'Poids lourds',16: 'Poids lourds',17: 'Poids lourds',

    # Véhicules agricoles / spéciaux : Engin spécial; Tracteur agricole; Engin de chantier
    20: 'Véhicules agricoles / spéciaux',21: 'Véhicules agricoles / spéciaux',50: 'Véhicules agricoles / spéciaux',

    # Transport en commun : Autobus; Autocar; Train routier; Tramway
    37: 'Transport en commun',38: 'Transport en commun',39: 'Transport en commun',40: 'Transport en commun',

    # Autres véhicules professionnels : Ambulance; Véhicule prioritaire (pompiers/police), Voiture de dépannage; Déneigeuse / voirie
    32: 'Véhicules professionnels spécialisés',41: 'Véhicules professionnels spécialisés',42: 'Véhicules professionnels spécialisés',
    43: 'Véhicules professionnels spécialisés',

    # Catégorie générique
    99: 'Autre'# Inconnu, non classé
})


In [218]:
base_final["catv"].value_counts()

catv
Véhicules légers                        81081
Deux-roues motorisés                    19221
Véhicules légers pro                     8529
Deux-roues non motorisés                 6476
Véhicules professionnels spécialisés     3345
Poids lourds                             3028
Véhicules agricoles / spéciaux           2127
Transport en commun                      1604
Autre                                     526
Aucun                                     136
Name: count, dtype: int64

In [219]:
# Équipement de sécurité (secu1)
base_final["secu1"] = base_final["secu1"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})

# Équipement de sécurité (secu2)
base_final["secu2"] = base_final["secu2"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})

# Équipement de sécurité (secu3)
base_final["secu3"] = base_final["secu3"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})


##### Sauvegarde de la 1ere base traitée

In [220]:
base_final.to_csv("base_finale.csv")

#### renommons nos variables qualitatives

In [221]:
base_f=base_final.copy()

In [222]:
# Recode des variables qualitatives selon le fichier officiel ONISR
# À appliquer à base_f (ou base_finale) après importation des données

# Labels pour le sexe
base_f["sexe"] = base_f["sexe"].replace({1: "Homme",2: "Femme"})

# Catégorie d’usager (catu)
base_f['catu'] = base_f['catu'].replace({1: 'Conducteur',2: 'Passager',3: 'Piéton'})

# Gravité (grav)
#base_f['grav'] = base_f['grav'].replace({1: 'Indemne',2: 'Tué',3: 'Blessé hospitalisé',4: 'Blessé léger'})

# Trajet (trajet)
base_f['trajet'] = base_f['trajet'].replace({1: 'Domicile-travail',2: 'Domicile-école',3: 'Courses-achats',4: 'Utilisation professionnelle',5: 'Promenade-loisirs',
    9: 'Autre'})

# Lumière (lum)
base_f['lum'] = base_f['lum'].replace({1: 'Plein jour', 2: 'Crépuscule ou aube',3: 'Nuit sans éclairage public',
    4: 'Nuit éclairage non allumé',5: 'Nuit éclairage allumé'})

# Conditions atmosphériques (atm)
base_f['atm'] = base_f['atm'].replace({1: 'Normale',2: 'Pluie légère',3: 'Pluie forte',4: 'Neige/grêle',5: 'Brouillard/fumée',6: 'Vent fort/tempête',
    7: 'Éblouissement',8: 'Temps couvert',9: 'Autre'})

# Manœuvre (manv)
base_f['manv'] = base_f['manv'].replace({1: 'Circulation normale',2: 'Circulation normale',3: 'Circulation normale',
    4: 'Marche arrière',5: 'Manœuvre complexe',6: 'Manœuvre complexe',7:'Dans le couloir du bus/MS',8:'Dans le couloir du bus/SI',9: 'Manœuvre complexe',10: 'Manœuvre complexe',11: 'Changement de voie',
    12: 'Changement de voie',13: 'Changement de voie',14: 'Changement de voie',15: 'Changement de voie',16: 'Changement de voie',
    17: 'Manœuvre complexe',18: 'Manœuvre complexe',19: 'Manœuvre complexe',20: 'Stationnement',21: 'Évitement',
    22: 'Ouverture de porte',23: 'Stationnement/Arrêt',24: 'Stationnement/Arrêt',25: 'Stationnement/Arrêt',
    26: 'Autre'
})

# Type de choc (choc)
base_f['choc'] = base_f['choc'].replace({1: 'Avant',2: 'Arrière',3: 'Latéral',4: 'Multichoc',5: 'Autre'
})

# Équipement de sécurité (secu1)
base_f["secu1"] = base_f["secu1"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})

# Équipement de sécurité (secu2)
base_f["secu2"] = base_f["secu2"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})

# Équipement de sécurité (secu3)
base_f["secu3"] = base_f["secu3"].replace({1: "Ceinture",2: "Casque",3: "Dispositif enfants",
     4: "Gilet réfléchissant",5: "Airbag",6: "Gants",7: "Gants + Airbag",8: "Non renseigné", 9: "Autre"})
# Lieu piéton (locp)
base_f['locp'] = base_f['locp'].replace({1: 'sur chaussée',2: 'sur chaussée',3: 'Sur passage piéton',
    4: 'Divers',5: 'Divers',6: 'Divers',7: 'Divers',8: 'Divers',9: 'Divers'})

# Action du piéton (actp)
base_f['actp'] = base_f['actp'].replace({1: 'Sens véhicule',2: 'Sens inverse véhicule',3: 'Traversant',4: 'Masqué',
    5: 'Jouant/courant',6: 'Avec animal',9: 'Autre','A': 'Monte/descend véhicule','B': 'Inconnue'})

# État du piéton (etatp)
base_f['etatp'] = base_f['etatp'].replace({1: 'Seul',2: 'Accompagné',3: 'En groupe'})


In [223]:
base_f['manv'].value_counts()

manv
Circulation normale          68552
Changement de voie           23720
Manœuvre complexe            13978
Aucun                         6621
Autre                         4827
Stationnement/Arrêt           4009
Évitement                     2070
Stationnement                  809
Marche arrière                 717
Ouverture de porte             409
Dans le couloir du bus/MS      316
Dans le couloir du bus/SI       45
Name: count, dtype: int64

In [224]:
base_f.to_csv("base_f.csv")